# Example: Train a Recommender

In this example, we want to guide you through the process of training a recommender from offline evaluations. You can then use this recommender to provide you with good default choices for models and hyperparameters on new datasets. All without ever training a model on that dataset!

## Initialize the Tracker

As usually, we first need to initialize a tracker to access the offline evaluations.

In [1]:
from pathlib import Path
from tsbench.evaluations.tracking import ModelTracker

In [2]:
tracker = ModelTracker.from_directory(Path.home() / "evaluations")

## Initialize the Surrogate

Using the tracker, we can easily train a surrogate model which learns from all performances available from the tracker. Here, we use a ranking MLP as it provides the best performance.

Further, we want the surrogate model to predict an accuracy metric (the nCRPS) and the latency. Thus, we can later obtain models from the recommender which are both accurate and fast.

In [3]:
from tsbench.surrogate import MLPSurrogate

In [4]:
surrogate = MLPSurrogate(
    tracker,
    objective="ranking",
    discount="linear",
    hidden_layer_sizes=[32, 32],
    predict=["ncrps_mean", "latency_mean"],
)

## Train the Recommender

The surrogate can subsequently be passed to a recommender which is responsible for the multi-objective selection. We focus the selection on the nCRPS such that the first recommended model optimizes for nCRPS, the second one for latency, and the third one provides a tradeoff.

In [5]:
from tsbench.recommender import ParetoRecommender

In [6]:
recommender = ParetoRecommender(
    surrogate, objectives=["ncrps_mean", "latency_mean"], focus="ncrps_mean"
)

Having initialized the recommender, we can fit it. We will use all available data since we won't evaluate the performance on a test set here. Note that, since we are using the MLP surrogate, training takes a few dozen seconds.

In [7]:
data = tracker.get_evaluations()

In [8]:
recommender.fit(data.configurations, data.performances)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/ubuntu/tsbench/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Predicting Default Models

Once the recommender is trained, we can use it to predict models for unseen datasets. While we are required to pass a dataset here, it will be ignored since the specific surrogate model that we used does not use dataset meta features for its prediction.

In [9]:
from tsbench.config import DATASET_REGISTRY

In [10]:
dummy_dataset = DATASET_REGISTRY["m4_monthly"]

In [11]:
recommendations = recommender.recommend(dummy_dataset, max_count=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/ubuntu/tsbench/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


As we can see, the first recommendation is a large DeepAR model (4 layers and 80 cells) which apparently tends to be very accurate.

In [12]:
recommendations[0].config

DeepARModelConfig(training_fraction=1.0, learning_rate=0.001, context_length_multiple=1, num_layers=4, num_cells=80)

The second recommendation is a Seasonal Naïve model, which is arguably very fast, independently of the dataset.

In [13]:
recommendations[1].config

SeasonalNaiveModelConfig()

The third recommendation is a Simple Feedforward model with 3 hidden layers and a non-standard context length multiple of 2.

In [14]:
recommendations[2].config

SimpleFeedforwardModelConfig(training_fraction=0.3333333333333333, learning_rate=0.001, context_length_multiple=2, hidden_dim=60, num_layers=3)

If you were to find a model for an unseen dataset, you could now train the first few recommendations of the recommender on your data to figure out which one actually performs best (in terms of latency and accuracy).